In [11]:
# Importar librerías
import os
import csv
import pandas as pd
import numpy as np
import zipfile
import requests
import io
import src.comercio
import warnings
import boto3

warnings.filterwarnings("ignore")

# Lectura del .zip página INEGI

In [2]:
PATH_GUESTS = "https://www.inegi.org.mx/contenidos/masiva/denue/esenciales/Actividades_Economicas_No_Esenciales_csv.zip"
# Descargar el archivo zip
response = requests.get(PATH_GUESTS)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))


In [3]:
# Crear una carpeta para almacenar los archivos extraídos
output_folder = "data_comercios/"  # Especifica la ruta de la carpeta de salida
os.makedirs(output_folder, exist_ok=True)

# Extraer todos los archivos CSV del ZIP
csv_files = []
for file_name in zip_file.namelist():
    if file_name.endswith('.csv'):
        csv_file = zip_file.extract(file_name, output_folder)
        csv_files.append(csv_file)


In [8]:
csv_files[0]

'data_comercios\\actividades_economicas_no_escenciales_csv\\th_denue_act_NOesenciales_ent_01_09.csv'

In [12]:
df_t = pd.DataFrame()
for i in range(len(csv_files)):
    df = src.comercio.lecture_data(csv_files[i]) 
    df_t = pd.concat([df_t, df])

b'Skipping line 280230: expected 42 fields, saw 43\n'
b'Skipping line 188790: expected 42 fields, saw 43\n'


In [13]:
df_t.head()

,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,...,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta,es_esencial
id,,,,,,,,,,,,,,,,,,,,,
6981122,EMBOBINADOS DAVILA,NaN,811312,Reparación y mantenimiento de maquinaria y equ...,1,CALLE,SALUD,PROLONGACION,AVENIDA POLIDUCTO,AVENIDA,...,2668,2,4492758040.0,EMBOBINADOSDAVILA@GMAIL.COM,NaN,NaN,21.869575,-102.243051,2019-11,0
6981121,DESECHABLES,NaN,465914,Comercio al por menor de artículos desechables,1,CALLE,SALUD,PROLONGACION,AVENIDA POLIDUCTO,AVENIDA,...,2668,2,4493692373.0,NaN,NaN,NaN,21.869584,-102.243079,2019-11,0
6981120,ROPA AMERICANA,NaN,465215,Comercio al por menor de artículos y aparatos ...,1,CALLE,SALUD,PROLONGACION,AVENIDA POLIDUCTO,AVENIDA,...,2668,2,4493863851.0,NaN,NaN,NaN,21.869693,-102.243406,2019-11,0
6981119,ACCESORIOS Y NOVEDADES,ACCESORIOS Y NOVEDADES,463213,Comercio al por menor de lencería,1,CALLE,SALUD,PROLONGACION,AVENIDA POLIDUCTO,AVENIDA,...,2668,2,4493890114.0,NaN,NaN,NaN,21.869895,-102.244008,2019-11,0
6981114,PENSION MADAGASCAR,NaN,812990,Otros servicios personales,1,CALLE,AURORA BOREAL,AVENIDA,MARIANO ESCOBEDO,CALLE,...,2475,20,4494485486.0,NaN,NaN,NaN,21.861681,-102.273371,2019-11,0


In [14]:
#Se guarda en Local el archivo
df_t.to_csv("salidas/denue.csv")

# S3

In [17]:
session = boto3.Session(profile_name='itam_datascientist')
s3 = session.client('s3')

In [18]:
# Crea un nuevo Bucket
# OJO: Aqui cambia el nombre del bucket, utiliza tu nombre como sufijo, para
# que sea único en el mundo, si no te va a salir error. Si tuviste éxito, el
# API de boto3 te va a regresar un HTTPStatusCode de 200. Eso es éxito.
BUCKET_NAME = "comercios-denue"
s3.create_bucket(Bucket=BUCKET_NAME)

{'ResponseMetadata': {'RequestId': 'BPP7CR9QYA25HSQP',
  'HostId': 'pljaRNg/FoEtLtKKh+Ot0BRgQp9fvMIUb5FPYw/MX3/+3vRYPZhs20lBdu6MvqPnv41m3pXm4SE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pljaRNg/FoEtLtKKh+Ot0BRgQp9fvMIUb5FPYw/MX3/+3vRYPZhs20lBdu6MvqPnv41m3pXm4SE=',
   'x-amz-request-id': 'BPP7CR9QYA25HSQP',
   'date': 'Mon, 22 May 2023 05:27:46 GMT',
   'location': '/comercios-denue',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/comercios-denue'}

In [19]:
# Upload los dos archivos a S3
s3.upload_file(Filename="salidas/denue.csv", Bucket=BUCKET_NAME, Key="salidas/denue.csv")